In [1]:
#
# 6/1/2021
# University of Oregon
# CIS 472: Machin Learning
# Instructor: Thien Huu Nguyen
#
# Final Project
# Part 3: tune HP
# Submit: Warren Liu
# 

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import kerastuner as kt

In [3]:
# Load data and rescale
fashion_mnist = keras.datasets.fashion_mnist
(train_x, train_y), (test_x, test_y) = fashion_mnist.load_data()
train_x, test_x = train_x / 255.0, test_x / 255.0
# Because this dataset contains only balck and white images,
# thus, add one more axis to it
train_x = train_x[..., tf.newaxis]
test_x = test_x[..., tf.newaxis]
inputShape = train_x.shape

In [4]:
def model_builder(hp):
    model = keras.Sequential()
    conv1_size = hp.Int('filters1', min_value=32, max_value=128, step=32)
    conv2_size = hp.Int('filters2', min_value=32, max_value=128, step=32)
    conv3_size = hp.Int('filters3', min_value=32, max_value=128, step=32)
    dense_size = hp.Int('units', min_value=32, max_value=128, step=32)
    model.add(layers.Conv2D(filters=conv1_size, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling2D((2,2), padding='same'))
    model.add(layers.Conv2D(filters=conv2_size, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling2D((2,2), padding='same'))
    model.add(layers.Conv2D(filters=conv3_size, kernel_size=3, activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(units=dense_size, activation='relu'))
    model.add(layers.Dense(10))

    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    return model

In [5]:
tuner = kt.Hyperband(model_builder,
        objective='val_accuracy',
        max_epochs=10,
        factor=3)

In [6]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(train_x, train_y, epochs=50, validation_split=0.2, callbacks=[stop_early])
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
print(f'''
    conv1 size: {best_hps.get('filters1')}\n
    conv2 size: {best_hps.get('filters2')}\n
    conv3 size: {best_hps.get('filters3')}\n
    dense size: {best_hps.get('units')}
''')

Trial 30 Complete [00h 01m 06s]
val_accuracy: 0.9141666889190674

Best val_accuracy So Far: 0.9211666584014893
Total elapsed time: 00h 13m 48s
INFO:tensorflow:Oracle triggered exit

    conv1 size: 64

    conv2 size: 128

    conv3 size: 96

    dense size: 32

